In [3]:
import pandas as pd
import gensim 
from gensim.models import Word2Vec 
from tqdm import tqdm
import re
import numpy as np
  
dataset = pd.read_csv("data/names_combined.csv")

dataset.dropna(inplace=True)
dataset.replace('b','black',inplace=True)
  
data = [] 
corpus = []

def features(name):
	name = re.sub(' ','',name)
	bigrams = []
	trigrams = []
	for i in range(len(name)-1):
		bigrams.append(name[i:i+2])
		if i < len(name) - 2:
			trigrams.append(name[i:i+3])
	trigrams.append(name[-3:])
	return bigrams+trigrams

for i in tqdm(range(len(dataset['name']))):
	temp = []
	review = re.sub('[^a-zA-Z]', ' ', dataset['name'].values[i])
	review = review.lower()
	temp = list(review) + features(review)
	data.append(temp)
	corpus.append(review)

100%|██████████| 120719/120719 [00:03<00:00, 35976.83it/s]


In [4]:
# Create & train Char vec model
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 350, window = 30) ## 300,10
print('model created')
model1.train(data,total_examples=model1.corpus_count, epochs=15)
print('model trained')

model created
model trained


In [41]:
y = dataset.iloc[:, 2].values
X = []
for c in tqdm(corpus):
	vecs = model1[list(c) + features(c)]
	X.append(vecs.sum(axis=0))
X = np.array(X)

  0%|          | 0/120719 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████| 120719/120719 [00:22<00:00, 5333.41it/s]


In [6]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X, y)

In [42]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1,1)).toarray()

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)#, random_state = 0)

In [9]:
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression(max_iter=40,random_state = 0, solver='sag',verbose=2,n_jobs=3)
# classifier.fit(X_train, y_train)
# y_pred = classifier.predict(X_test)

# # Making the Confusion Matrix
# from sklearn.metrics import confusion_matrix, classification_report
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
# print(classification_report(y_test, y_pred,target_names=classifier.classes_))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


max_iter reached after 48 seconds
max_iter reached after 48 seconds
max_iter reached after 48 seconds


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=3)]: Done   2 out of   4 | elapsed:   48.1s remaining:   48.1s


max_iter reached after 22 seconds


[Parallel(n_jobs=3)]: Done   4 out of   4 | elapsed:  1.2min finished


[[7058  420   98 2241]
 [ 285 8875   53  567]
 [ 102   60 9521   59]
 [2957 1613  124 4883]]
              precision    recall  f1-score   support

       black       0.68      0.72      0.70      9817
    hispanic       0.81      0.91      0.86      9780
      indian       0.97      0.98      0.97      9742
       white       0.63      0.51      0.56      9577

   micro avg       0.78      0.78      0.78     38916
   macro avg       0.77      0.78      0.77     38916
weighted avg       0.77      0.78      0.77     38916



In [44]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, LSTM, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers

classifier = Sequential()

######## race classification 87% test accuracy
classifier.add(Dense(input_dim=350, output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
#classifier.add(Dropout(0.2))  # Dropout overfitting
classifier.add(Dense(output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=64, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=64, activation='relu'))
classifier.add(BatchNormalization())
#classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=32, activation='relu'))
classifier.add(BatchNormalization())
#classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=4, activation='softmax'))

##### gender classification model 98% test acc
# classifier1.add(Dense(input_dim=300, output_dim=256, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))  # Dropout overfitting
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=32, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=2, activation='softmax'))


classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
history = classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=256)

classifier1.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
history1 = classifier1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=256)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=350, activation="relu", units=256)`
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykern

Train on 96575 samples, validate on 24144 samples
Epoch 1/10
96575/96575 [==============================] - 11s 116us/step - loss: 0.6394 - acc: 0.6932 - val_loss: 0.5302 - val_acc: 0.7457
Epoch 2/10
96575/96575 [==============================] - 8s 87us/step - loss: 0.4973 - acc: 0.7587 - val_loss: 0.4979 - val_acc: 0.7551
Epoch 3/10
96575/96575 [==============================] - 7s 76us/step - loss: 0.4726 - acc: 0.7708 - val_loss: 0.4875 - val_acc: 0.7573
Epoch 4/10
96575/96575 [==============================] - 7s 72us/step - loss: 0.4560 - acc: 0.7799 - val_loss: 0.4636 - val_acc: 0.7769
Epoch 5/10
96575/96575 [==============================] - 7s 73us/step - loss: 0.4436 - acc: 0.7879 - val_loss: 0.4634 - val_acc: 0.7749
Epoch 6/10
96575/96575 [==============================] - 7s 76us/step - loss: 0.4349 - acc: 0.7922 - val_loss: 0.4615 - val_acc: 0.7787
Epoch 7/10
96575/96575 [==============================] - 7s 74us/step - loss: 0.4235 - acc: 0.7991 - val_loss: 0.4695 - val_a

In [45]:
def predict_gen(name):
    classes = ['female','male']
    namevec = model1[list(name)+features(name)].sum(axis=0).flatten()
    pred = classifier.predict(namevec.reshape(1,350))
    predprob = pred[0][np.argmax(pred)]*100
    gender = classes[np.argmax(pred)]
    return f'{name} is probably a {gender}.... chances:{predprob}%'

def predict_race(name):
    classes = ['black','hispanic','indian','white']
    namevec = model1[list(name)+features(name)].sum(axis=0).flatten()
    pred = classifier.predict(namevec.reshape(1,350))
    predprob = pred[0][np.argmax(pred)]*100
    race = classes[np.argmax(pred)]
    return pred
    return f'{name} is probably {race}.... chances:{predprob}%'

In [56]:
predict_race('andres')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


array([[4.1473142e-10, 1.9257501e-05, 9.9998069e-01, 3.1888163e-09]],
      dtype=float32)